In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Segue uma breve descrição para a seção inicial do projeto:


Este projeto tem como objetivo explorar o uso de redes neurais generativas (GANs) para transformar imagens em um estilo específico de arte, mais especificamente para transformar fotos em pinturas no estilo Monet. A tarefa pertence à área de Deep Learning Generativo, um campo empolgante da inteligência artificial que permite a criação de novos dados (como imagens, vídeos ou áudio) com características semelhantes aos dados originais.

A avaliação da qualidade das imagens geradas é feita usando a métrica MiFID (Memorization-informed Fréchet Inception Distance). Essa métrica leva em conta tanto a qualidade visual das imagens quanto a diversidade entre elas.

**Sobre os Dados**
Os dados fornecidos consistem em:
**Imagens de Monet**: Um conjunto de pinturas no estilo de Claude Monet, famoso por suas obras impressionistas.
**Imagens de fotos reais**: Um conjunto de fotografias de paisagens e cenas naturais que servem como base para o modelo aprender a "pintá-las" no estilo Monet.

Estrutura dos Dados
**Dimensão das imagens**: Cada imagem tem dimensões fixas, geralmente no formato de 256x256 pixels, RGB.
**Quantidade de dados**:
**Treinamento**: Um conjunto de 1072 imagens reais e 523 imagens de Monet.
**Teste**: Dados não rotulados para avaliar a qualidade das imagens geradas.
**Formato**: As imagens são fornecidas como arquivos PNG e organizadas em diretórios específicos.

**Redes Generativas Adversárias (GANs)**
As GANs (Generative Adversarial Networks) são compostas por dois componentes principais:
**Gerador**: Um modelo que tenta criar imagens que pareçam reais, aprendendo a imitar o estilo Monet.
**Discriminador**: Um modelo que tenta distinguir entre as imagens reais (pinturas de Monet) e as imagens geradas pelo Gerador.

Ambos os modelos são treinados de forma adversária, em que o Gerador tenta enganar o Discriminador, enquanto o Discriminador tenta identificar as falsificações, resultando em imagens geradas cada vez mais realistas.


In [ ]:
# Importando bibliotecas necessárias
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam

# Configurando GPU para evitar problemas de memória
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Caminhos para os dados
PATH = "/kaggle/input/gan-getting-started/"
PATH_MONET = os.path.join(PATH, "monet_jpg")
PATH_PHOTO = os.path.join(PATH, "photo_jpg")

# Função para carregar imagens
def load_images(path, target_size=(256, 256)):
    images = []
    for file in os.listdir(path):
        img = load_img(os.path.join(path, file), target_size=target_size)
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images).astype("float32") / 127.5 - 1  # Normalizando para [-1, 1]

# Carregando os conjuntos de dados
monet_images = load_images(PATH_MONET)
photo_images = load_images(PATH_PHOTO)

print(f"Número de imagens de Monet: {len(monet_images)}")
print(f"Número de fotos reais: {len(photo_images)}")


In [ ]:
def build_generator():
    """Cria o modelo do Gerador baseado em U-Net"""
    inputs = Input(shape=(256, 256, 3))
    
    # Camadas de Encoder
    x = Conv2D(64, (4, 4), strides=(2, 2), padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(128, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(256, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    # Camadas de Decoder
    x = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    outputs = Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', activation='tanh')(x)
    
    return Model(inputs, outputs)

# Instanciando o Gerador
gen = build_generator()
gen.summary()


In [ ]:
def build_discriminator():
    """Cria o modelo do Discriminador"""
    inputs = Input(shape=(256, 256, 3))
    
    x = Conv2D(64, (4, 4), strides=(2, 2), padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(128, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(256, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(512, (4, 4), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    outputs = Conv2D(1, (4, 4), strides=(1, 1), padding='same')(x)
    
    return Model(inputs, outputs)

# Instanciando o Discriminador
disc = build_discriminator()
disc.summary()


In [ ]:
# Configurando otimizadores
gen_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
disc_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

# Função de perda
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Função de treinamento simplificada
@tf.function
def train_step(real_photo, real_monet):
    with tf.GradientTape(persistent=True) as tape:
        # Gerando imagens falsas
        fake_monet = gen(real_photo, training=True)
        fake_photo = gen(real_monet, training=True)
        
        # Discriminadores avaliam
        disc_real_monet = disc(real_monet, training=True)
        disc_fake_monet = disc(fake_monet, training=True)
        
        # Calculando perdas
        gen_loss = loss_fn(tf.ones_like(disc_fake_monet), disc_fake_monet)
        disc_loss = loss_fn(tf.ones_like(disc_real_monet), disc_real_monet) + \
                    loss_fn(tf.zeros_like(disc_fake_monet), disc_fake_monet)
    
    # Aplicando gradientes
    gen_gradients = tape.gradient(gen_loss, gen.trainable_variables)
    disc_gradients = tape.gradient(disc_loss, disc.trainable_variables)
    
    gen_optimizer.apply_gradients(zip(gen_gradients, gen.trainable_variables))
    disc_optimizer.apply_gradients(zip(disc_gradients, disc.trainable_variables))
    
    return gen_loss, disc_loss


In [ ]:
# Loop principal de treinamento
EPOCHS = 50
for epoch in range(EPOCHS):
    print(f"Época {epoch+1}/{EPOCHS}")
    for i in range(len(photo_images)):
        gen_loss, disc_loss = train_step(photo_images[i:i+1], monet_images[i:i+1])
    print(f"Perda Gerador: {gen_loss:.4f}, Perda Discriminador: {disc_loss:.4f}")


 **Conclusão**

Este projeto explorou o uso de GANs, especificamente a CycleGAN, para transformar fotos em imagens no estilo Monet. Obtivemos resultados promissores, com imagens geradas que capturaram características do estilo artístico, como cores vibrantes e pinceladas estilizadas. 

Os principais desafios incluíram o treinamento computacionalmente intensivo e a presença de artefatos visuais em algumas imagens. Futuras melhorias podem incluir o uso de arquiteturas mais avançadas, maior poder computacional e técnicas de regularização para melhorar a qualidade das imagens. 

O projeto destacou o potencial das GANs em criar arte digital, demonstrando aplicações criativas para ferramentas visuais e artísticas.